<a href="https://colab.research.google.com/github/MacGyver2016/outreach-twin/blob/main/Fundraising_and_Recruitment_Outreach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
# Install packages
!pip install -q crewai #openai langchain beautifulsoup4 requests
!pip install 'crewai[tools]'

In [ ]:
# Import libraries
from crewai_tools import DallETool, SerperDevTool
import os
from crewai import LLM, Agent, Task, Crew, Process

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [ ]:
# Set API keys
# IMPORTANT: User needs to input their own API keys here
os.environ["GEMINI_API_KEY"] = "<YOUR_GEMINI_API_KEY>"
os.environ["SERPER_API_KEY"] = "<YOUR_SERPER_API_KEY>"
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_API_KEY>"

In [ ]:
# Input important company information (or use defaults)
"""
    This information will be used to personalize outreach efforts so that they align with the company

    company_name: common name of the company
    company_mission: either the company's mission or a summary of it
    company_participant_number: approximatley how many people are served by the company plus how many people work within the company
    company_region: where those who are served by the company reside
    company_results_desired: how many sponsors/colleges you'd like in the output
    company_event: the event the company is organizing
"""
company_name = "Hacknation"
company_mission = "innovate for impact and build a global community of hackers, push the boundaries of AI, accelerate AI learning in a fun collaborative way, and build bridges between industry, academia, and the public sector"
company_participant_number = 2000
company_region = "global"
company_results_desired = 2
company_event = "virtual ai hackathon"

# company_name = input("Please input your company's name:")
# company_mission = input("Please summarize your company's mission:")
# company_participant_number = int(input("Please input how many people you serve:"))
# company_region = input("Please input where your participants are located:")
# company_results_desired = int(input("Please input how many results you would like to see:"))
# company_event = input("Please describe the event you are organizing:")

In [ ]:
# Instantiate tools
"""
    search_tool: allows the ai crew to search the internet for relevant information using the Serper API
    dalle_tool: allows the ai crew to generate images using the Dall-E model
"""
search_tool = SerperDevTool()
dalle_tool = DallETool(model="dall-e-3", size="1024x1024", quality="standard", n=1)

# Setup LLM
"""Gemini is the default, but a different model, such as OpenAI may be substituted here"""
llm = LLM(model="gemini/gemini-2.0-flash", temperature=0.7)

## Sponsorship

In [ ]:
# Define Sponsorship Crew Agents
"""
    Defines agents with specialized roles to work together to create a sponsorship plan.

    The general roles are as follows:
    sponsorship_prospect_researcher: identifies sponsors
    sponsorship_outreach_specialist: creates communications for sponsors
    sponsorship_manager: combines all that is done by the other agents to create a final plan
"""
sponsorship_prospect_researcher = Agent(
    role="Prospect Researcher",
    goal=f"Identify potentential sponsors for {company_name}",
    backstory=f"You are a skilled prospect researcher who has experience matching companies to potential sponsors in {company_region}",
    llm=llm,
    tools=[search_tool],
    verbose=True,
    allow_delegation=False,
)

sponsorship_outreach_specialist = Agent(
    role="Outreach Specialist",
    goal=f"Secure sponsors for {company_name} through effective email communication",
    backstory="You are an effective writer who is both professional, but also fun and great at grabbing attention.",
    llm=llm,
    tools=[search_tool], # add dalle_tool here if you have credits
    verbose=True,
    allow_delegation=False,
)

sponsorship_manager = Agent(
    role="Outreach Manager",
    goal="Combine all aspects to create an outreach plan.",
    backstory="You have managed outreach teams for medium sized companies for many years. This outreach should help this company grow significantly.",
    llm=llm,
    tools=[search_tool],
    verbose=True,
    allow_delegation=True
)

In [ ]:
# Define Sponsorship Crew Tasks
"""
    Defines tasks for agents to complete with the end goal of creating a sponsorship plan.

    find_sponsors_task: identifies potential sponsors
    find_sponsor_contact_info_task: finds contact methods (preferably email addresses) for each sponsor identified
    determine_sponsor_attitude_task: determines amount to ask for and the best tone to use in email outreach
    write_sponsor_email_task: writes an outreach email for each potential sponsor
    create_sponsor_image_task: creates an image for each potential sponsor
    sponsorship_proposal: creates a detailed sponsorship proposal document that is general enough to apply to each company in the list
    sponsor_final_output_task: organizes all of the information from the previous tasks to create a plan for the company to procure sponsors
"""
find_sponsors_task = Task(
    description=f"Find the {company_results_desired} best sponsors for {company_name}, whose company mission aligns with {company_name}'s mission of {company_mission}, are located in {company_region}, and could feasibly sponsor an event for {company_participant_number} people",
    agent=sponsorship_prospect_researcher,
    expected_output=f"A bullet-point list of potential sponsors, including their locations, why their mission aligns with {company_name}'s mission, and why this sponsorship would benefit the sponsor")

find_sponsor_contact_info_task = Task(
    description="For each company in the list, find a viable email to use to contact the company to ask for their sponsorship",
    agent=sponsorship_prospect_researcher,
    context=[find_sponsors_task],
    expected_output="A list with contact information for each company from the sponsor list"
)

determine_sponsor_attitude_task = Task(
    description="Using the sponsor list, determine the attitude of each potentential sponsor based on their website and social media.",
    agent=sponsorship_outreach_specialist,
    tools=[search_tool],
    context=[find_sponsors_task],
    expected_output= f"A bullet-point list that lists each potential sponsor, how much {company_name} should ask for in their request (keeping in mind that they serve {company_participant_number} people), and what attitude should be used when approaching the sponsor to get the best feedback",
)

write_sponsor_email_task = Task(
    description="Using the lists, write an outreach email for each potential sponsor.",
    agent=sponsorship_outreach_specialist,
    tools=[search_tool],
    context=[find_sponsors_task, determine_sponsor_attitude_task],
    expected_output="Text for an outreach email for each company"
)

create_sponsor_image_task = Task(
    description=f"Create a visual that will convince each sponsor to partner with {company_name} that will accompany the written email",
    agent=sponsorship_outreach_specialist,
    context=[find_sponsors_task, determine_sponsor_attitude_task, write_sponsor_email_task],
    expected_output="An image for each sponsor"
)

sponsorship_proposal = Task(
    description="Create a detailed sponsorship proposal document that is general enough to apply to each company in the list",
    agent=sponsorship_manager,
    context=[find_sponsors_task, determine_sponsor_attitude_task, write_sponsor_email_task, create_sponsor_image_task, find_sponsor_contact_info_task],
    expected_output="A detailed sponsorship proposal document"
)

sponsor_final_output_task = Task(
    description=f"Organize all of the information from the previous tasks to create a plan for {company_name} to procure sponsors",
    agent=sponsorship_manager,
    context=[find_sponsors_task, determine_sponsor_attitude_task, write_sponsor_email_task, create_sponsor_image_task, find_sponsor_contact_info_task],
    expected_output="A final list, including the previously written sponsorship proposal, emails, visuals, contact information, and any other important information for each company"
)



In [ ]:
# Define Sponsorship Crew
sponsorship_crew = Crew(
    agents=[sponsorship_prospect_researcher, sponsorship_outreach_specialist, sponsorship_manager],
    tasks=[find_sponsors_task, find_sponsor_contact_info_task, determine_sponsor_attitude_task, write_sponsor_email_task, create_sponsor_image_task, sponsorship_proposal, sponsor_final_output_task],
    verbose=True,
    max_rpm = 10,   # Gemini allows 10 requests per minute for free. Change this if you are allowed more requests per minute.
)

In [ ]:
# Run Sponsorship Crew
"""
    Starts the crew running.
    As agents complete tasks, the output will appear below
    The final printed output will have a detailed sponsorship plan for each company
"""
result = sponsorship_crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7e55cc7d-6aa5-431f-bae6-d1cf46aa2b2b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Prospect Researcher                                                                                     │
│                                                                                                                 │
│  Task: Find the 2 best sponsors for Hacknation, whose company mission aligns with Hacknation's mission of       │
│  innovate for impact and build a global community of hackers, push the boundaries of AI, accelerate AI          │
│  learning in a fun collaborative way, and build bridges between industry, academia, and the public sector, are  │
│  located in global, and could feasibly sponsor an event for 2000 people                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']

...

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: ca1fd117-3b4b-4c79-8044-caf42135be8e                                                                     │
│  Agent: Prospect Researcher                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 7e55cc7d-6aa5-431f-bae6-d1cf46aa2b2b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ValueError: Invalid response from LLM call - None or empty.

## Participant

In [ ]:
# Define Participant Recruitment Crew Agents
"""
    Defines agents with specialized roles to work together to create a recruitment plan to colleges to increase participation.

    The general roles are as follows:
    participant_recruiter: identifies colleges
    participant_outreach_specialist: creates communications for colleges
    participant_outreach_manager: combines all that is done by the other agents to create a final plan
"""
participant_recruiter = Agent(
    role="Participant Recruiter",
    goal=f"Identify potentential colleges that could have participants for {company_name}'s {company_event} event",
    backstory=f"You are a skilled recruiter who has experience finding participants for {company_event} events in {company_region}",
    llm=llm,
    tools=[search_tool],
    verbose=True,
    allow_delegation=False,
)

participant_outreach_specialist = Agent(
    role="Participant Outreach Specialist",
    goal=f"Secure participants for {company_name} through effective email communication with their colleges",
    backstory="You are an effective writer who is both professional, but also fun and great at grabbing attention.",
    llm=llm,
    tools=[search_tool],
    verbose=True,
    allow_delegation=False,
)

participant_recruitment_manager = Agent(
    role="Recruitment Manager",
    goal="Combine all aspects to create an recruitment plan.",
    backstory="You have managed recruitment teams for medium sized events for many years. This outreach should help this company's participation grow significantly.",
    llm=llm,
    tools=[search_tool],
    verbose=True,
    allow_delegation=True
)

In [15]:
# Define Recruitment Crew Tasks
"""
    Defines tasks for agents to complete with the end goal of creating a recruitment outreach plan.

    find_colleges_task: identifies potential colleges to recruit participants from
    find_college_contact_info_task: finds contact methods (preferably email addresses) for each college identified
    determine_college_attitude_task: determines the best tone to use in email outreach
    write_recruitment_email_task: writes an outreach email for each college
    recruitment_final_output_task: organizes all of the information from the previous tasks to create a plan for the company to recruit participants
"""
find_colleges_task = Task(
    description=f"Find the {company_results_desired} best colleges for {company_name}, whose mission aligns with {company_name}'s mission of {company_mission}, and are located in {company_region}",
    agent=participant_recruiter,
    expected_output=f"A bullet-point list of potential colleges, including their locations, why their mission aligns with {company_name}'s mission, and why having their students participate in the event would benefit the college"
)

find_college_contact_info_task = Task(
    description="For each college in the list, find a viable email to use to contact the college to ask for them to tell their students about the event",
    agent=participant_recruiter,
    context=[find_colleges_task],
    expected_output="A list with contact information for each college from the college list"
)

determine_college_attitude_task = Task(
    description="Using the college list, determine the attitude of each college based on their website and social media.",
    agent=participant_outreach_specialist,
    tools=[search_tool],
    context=[find_colleges_task],
    expected_output= f"A bullet-point list that lists each college and what attitude should be used when approaching the college to get the best feedback",
)

write_recruitment_email_task = Task(
    description="Using the lists, write an recruitment email for each college.",
    agent=sponsorship_outreach_specialist,
    tools=[search_tool],
    context=[find_colleges_task, determine_college_attitude_task],
    expected_output="Text for an recruitment email for each college. This should include an introduction section addressed to the contact, as well as a paragraph or two that you are requesting be included in an email, sent by the contact, to the students"
)

recruitment_final_output_task = Task(
    description=f"Organize all of the information from the previous tasks to create a plan for {company_name} to recruit college students as participants",
    agent=sponsorship_manager,
    context=[find_colleges_task, find_college_contact_info_task, determine_college_attitude_task, write_recruitment_email_task],
    expected_output="A final list, including the previously written emails, contact information, and any other important information for each college"
)

In [16]:
# Define Recruitment Crew
recruitment_crew = Crew(
    agents=[participant_recruiter, participant_outreach_specialist, participant_recruitment_manager],
    tasks=[find_colleges_task, find_college_contact_info_task, determine_college_attitude_task, write_recruitment_email_task, recruitment_final_output_task],
    verbose=True,
    max_rpm = 10,   # Gemini allows 10 requests per minute for free. Change this if you are allowed more requests per minute.
)

In [17]:
# Run Recruitment Crew
"""
    Starts the crew running.
    As agents complete tasks, the output will appear below
    The final printed output will have a detailed recruitment plan for each college
"""
result = recruitment_crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ca18e614-b5bb-4a06-9bb4-eee84ddf6af9                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Participant Recruiter                                                                                   │
│                                                                                                                 │
│  Task: Find the 2 best colleges for Hacknation, whose mission aligns with Hacknation's mission of innovate for  │
│  impact and build a global community of hackers, push the boundaries of AI, accelerate AI learning in a fun     │
│  collaborative way, and build bridges between industry, academia, and the public sector, and are located in     │
│  global                                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Outreach Manager                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  **Hacknation College Outreach Plan: MIT & Stanford**                                                           │
│                                                                                                                 │
│  **I. Goal:** Recruit college students from MIT and Stanford to participate in Hacknation's global AI           │
│  hackathon.                                                                                                     │
│                                                                                                                 │
│  **II. Target Colleges:**                                                                                       │
│                                                                                                                 │
│  *   Massachusetts Institute of Technology (MIT), Cambridge, Massachusetts, USA                                 │
│  *   Stanford University, Stanford, California, USA                                                             │
│                                                                                                                 │
│  **III. Key Contacts & Communication Styles:**                                                                  │
│                                                                                                                 │
│  *   **MIT:**                                                                                                   │
│      *   Contact Email: grad-ap@eecs.mit.edu                                                                    │
│      *   Communication Style: Direct, data-driven, focused on innovation and impact.                            │
│  *   **Stanford:**                                                                                              │
│      *   Contact Email: hai-institute@stanford.edu                                                              │
│      *   Communication Style: Collaborative, mission-aligned, emphasizing social good and global community.     │
│                                                                                                                 │
│  **IV. Student Organizations (Targeted Outreach):**                                                             │
│                                                                                                                 │
│  *   **MIT:**                                                                                                   │
│      1.  AI@MIT: [https://www.ai-at-mit.com/](https://www.ai-at-mit.com/)                                       │
│      2.  MIT IEEE/ACM Club: (Check MIT EECS student groups page for link)                                       │
│      3.  MIT Sloan AI & ML Club: [https://sloangroups.mit.edu/AIML/](https://sloangroups.mit.edu/AIML/)         │
│  (Contact: Via website)                                                                                         │
│      4.  MIT+Harvard Women in AI: [https://mitharvardwai.github.io/](https://mitharvardwai.github.io/)          │
│      5.  Artificial Intelligence Club at MIT (Engage@MIT):                                                      │
│  [https://engage.mit.edu/club_signup?category_tags=6788

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f5adeed7-cd60-48ee-ad06-db33449605f8                                                                     │
│  Agent: Outreach Manager                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ca18e614-b5bb-4a06-9bb4-eee84ddf6af9                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```                                                                                              │
│  **Hacknation College Outreach Plan: MIT & Stanford**                                                           │
│                                                                                                                 │
│  **I. Goal:** Recruit college students from MIT and Stanford to participate in Hacknation's global AI           │
│  hackathon.                                                                                                     │
│                                                                                                                 │
│  **II. Target Colleges:**                                                                                       │
│                                                                                                                 │
│  *   Massachusetts Institute of Technology (MIT), Cambridge, Massachusetts, USA                                 │
│  *   Stanford University, Stanford, California, USA                                                             │
│                                                                                                                 │
│  **III. Key Contacts & Communication Styles:**                                                                  │
│                                                                                                                 │
│  *   **MIT:**                                                                                                   │
│      *   Contact Email: grad-ap@eecs.mit.edu                                                                    │
│      *   Communication Style: Direct, data-driven, focused on innovation and impact.                            │
│  *   **Stanford:**                                                                                              │
│      *   Contact Email: hai-institute@stanford.edu                                                              │
│      *   Communication Style: Collaborative, mission-aligned, emphasizing social good and global community.     │
│                                                                                                                 │
│  **IV. Student Organizations (Targeted Outreach):**                                                             │
│                                                                                                                 │
│  *   **MIT:**                                                                                                   │
│      1.  AI@MIT: [https://www.ai-at-mit.com/](https://www.ai-at-mit.com/)                                       │
│      2.  MIT IEEE/ACM Club: (Check MIT EECS student groups page for link)                                       │
│      3.  MIT Sloan AI & ML Club: [https://sloangroups.mit.edu/AIML/](https://sloangroups.mit.edu/AIML/)         │
│  (Contact: Via website)                                                                                         │
│      4.  MIT+Harvard Women in AI: [https://mitharvardwai.github.io/](https://mitharvardwai.github.io/)          │
│      5.  Artificial Intelligence Club at MIT (Engage@M

```
**Hacknation College Outreach Plan: MIT & Stanford**

**I. Goal:** Recruit college students from MIT and Stanford to participate in Hacknation's global AI hackathon.

**II. Target Colleges:**

*   Massachusetts Institute of Technology (MIT), Cambridge, Massachusetts, USA
*   Stanford University, Stanford, California, USA

**III. Key Contacts & Communication Styles:**

*   **MIT:**
    *   Contact Email: grad-ap@eecs.mit.edu
    *   Communication Style: Direct, data-driven, focused on innovation and impact.
*   **Stanford:**
    *   Contact Email: hai-institute@stanford.edu
    *   Communication Style: Collaborative, mission-aligned, emphasizing social good and global community.

**IV. Student Organizations (Targeted Outreach):**

*   **MIT:**
    1.  AI@MIT: [https://www.ai-at-mit.com/](https://www.ai-at-mit.com/)
    2.  MIT IEEE/ACM Club: (Check MIT EECS student groups page for link)
    3.  MIT Sloan AI & ML Club: [https://sloangroups.mit.edu/AIML/](https://sloangroups.mit.edu/A